In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import uproot

import numpy as np
import pandas as pd
import awkward as ak
import time
import matplotlib.pyplot as plt

from utils.datasets import *
from utils.training import *
from models.models import *

/opt/homebrew/Caskroom/miniforge/base/envs/EgammaDNN/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Data Preprocessing

root = "/Users/markmatthewman/Projects/PhD/EgammaDNN/data/GenSim/TICLv4_Mustache/electron"
fname = "HLTAnalyzerTree_IDEAL_Flat_train.root"
tree = "egRegDataEcalHLTV1"
batches = 256

dataset = RegressionDataset(root,fname,tree)
dataloader = DataLoader(dataset,batch_size=batches,shuffle=True)

In [3]:
# Set Trainer 

lr = 0.001
device = torch.device("cpu")

# Select Model
model = RegressionDNN(7)
model.to(device)

# Select Loss function
loss_fn = nn.MSELoss()

# Select optimiser
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

# Initialize Trainer
trainer = Trainer(model, loss_fn, dataloader, optimizer)


In [14]:

for batch, data_batch in tqdm(enumerate(dataloader), total=len(dataloader)):
    #data_batch_device = {key: val.to(self.device) for key, val in data_batch.items()}
    inpt = data_batch["features"].to(device)
    tgt = data_batch["targets"].to(device)
    #print(tgt.shape)
    d = model(inpt)
    logprob = model.log_prob(tgt,inpt)
    loss = -logprob.mean()

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 10 == 0:
        tqdm.write(f"loss: {loss.item():>7f}")

  0%|                                         | 11/3802 [00:00<01:12, 52.16it/s]

loss: -1.270175
loss: -1.230481


  1%|▎                                        | 32/3802 [00:00<01:00, 62.20it/s]

loss: -1.190065
loss: -1.290922


  1%|▌                                        | 53/3802 [00:00<00:59, 63.54it/s]

...

loss: -1.464279
loss: -1.518256


100%|██████████████████████████████████████▉| 3792/3802 [00:54<00:00, 69.60it/s]

loss: -1.456773
loss: -1.559768


100%|███████████████████████████████████████| 3802/3802 [00:55<00:00, 68.95it/s]

loss: -1.409008


In [8]:
data_batch = next(iter(dataloader))

In [9]:
inpt = data_batch["features"]
tgt = data_batch["targets"]

In [10]:
d = model(inpt)

In [11]:
d.mu

tensor([0.7064, 2.2544, 2.2564, 2.2541, 2.2627, 2.2435, 2.2705, 0.5148, 2.2448,
        0.1772, 2.2395, 2.2401, 2.2464, 2.2442, 2.2435, 0.7989, 2.2387, 2.2446,
        2.2517, 2.2487, 0.9486, 0.7595, 2.2322, 2.2416, 2.2510, 2.2430, 0.6918,
        2.2359, 2.2512, 2.2553, 2.2435, 2.2503, 2.2428, 0.9796, 2.2392, 2.2483,
        0.5863, 2.2623, 2.2413, 0.6864, 2.2427, 0.8264, 0.7166, 0.4156, 2.2610,
        0.7752, 2.2420, 2.2393, 2.2407, 2.2468, 0.4041, 2.2654, 2.2444, 2.2573,
        2.2388, 2.2430, 2.2565, 2.2513, 0.5737, 2.2416, 2.2476, 0.8724, 2.2397,
        2.2695, 2.2613, 2.2442, 2.2552, 2.2473, 2.2567, 2.2641, 2.2338, 2.2373,
        2.2402, 2.2471, 2.2434, 0.7695, 2.2495, 2.2525, 0.8694, 2.2558, 2.2365,
        2.2533, 0.2950, 2.2398, 2.2404, 2.2522, 2.2449, 2.2491, 2.2422, 2.2593,
        2.2404, 2.2415, 2.2525, 0.7305, 2.2520, 0.6263, 2.2403, 0.6599, 2.2572,
        2.2516, 0.3333, 2.2397, 0.5167, 0.5013, 1.0876, 2.2510, 2.2388, 0.3587,
        2.2505, 0.7153, 2.2483, 2.2584, 

In [12]:
d.log_prob(tgt)

tensor([[-1.4748, -2.4889, -2.4778,  ..., -1.4939, -1.4503, -2.4499],
        [-1.4488, -2.5473, -2.5356,  ..., -1.4689, -1.4237, -2.5082],
        [-1.4333, -2.5792, -2.5671,  ..., -1.4540, -1.4079, -2.5400],
        ...,
        [-1.4611, -2.5204, -2.5090,  ..., -1.4808, -1.4363, -2.4813],
        [-1.4396, -2.5663, -2.5543,  ..., -1.4602, -1.4144, -2.5271],
        [-1.4492, -2.5463, -2.5346,  ..., -1.4694, -1.4242, -2.5072]],
       grad_fn=<IndexPutBackward0>)

In [3]:
# Training
epochs = 1
trainer.full_train(epochs)

########## Epoch 0


  0%|                                                  | 0/3802 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniforge/base/envs/EgammaDNN/lib/python3.12/site-packages/torch/distributions/distribution.py:57: UserWarning: <class 'models.double_sided_crystal_ball.DoubleSidedCrystalBall'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(
  0%|                                          | 3/3802 [00:00<02:10, 29.02it/s]

torch.Size([256, 1])
loss:     nan
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])


  0%|▏                                        | 13/3802 [00:00<00:54, 69.39it/s]

loss:     nan
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])


...

 88%|█████████████████████████████████▎    | 3336/3802 [00:29<00:04, 110.81it/s]

torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
loss:     nan
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
loss:     nan
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])


 88%|█████████████████████████████████▌    | 3359/3802 [00:29<00:03, 112.98it/s]


torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
loss:     nan
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])
torch.Size([256, 1])



KeyboardInterrupt

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10579b650>>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/EgammaDNN/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

outdir = "saved_models/v1"
create_dir(outdir)
fname = "v1"
trainer.save(os.path.join(outdir,fname))